## Problem Set 6: TM

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2021.
<br>
Burton Rosenberg.
<br>
<br>
created: 6 April 2021
<br>last update: 6 April 2021

Jeffrey Hudak
***


### TuringMachine class

In [2]:
import string
import sys
import os
import argparse
import re
from itertools import product


#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 22 mar 2020
#    16 mar 2021, updated 
#     3 apr 2021, return conventions for accept/reject
#                 verbose_levels reimplemented
#                 character # is not allowed as a tape symbol
#                 for magical reasons, then " is also not allowed
#                 added class method help()
#                 
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#

# GRAMMAR for the TM description

# Comments (not shown in BNF) begin with a hash # and continue to the end
#    of the line
# The ident tokens are states
# The symbol tokens are tape symbolss
# The StateTransition semantics is:
#     tape_symbol_read tape_symbol_written action new_state
# The underscore _ is a tape blank:
# The : in a transition rule is the default tape symbol match when there is no
#    exactly matching transition rule; in the target section of the rule it 
#    is the value of the matchined tape symbol.

# A missing transition is considered a reject, not an error

class TuringMachine:
    
    verbose_levels = {'none':0,'verbose':1,'explain':2, 'debug':3}
    result_reasons = ['ok', 'transition missing', 'time limit']

    grammar = """
    M-> (Stanza [emptyline])*
    Stanza-> StartStanza | AcceptStanza | RejectStanza | StateStanze
    StartStanza-> "start" ":" ident
    AcceptStanza-> "accept" ":" ident ([newline] [indent] ident])*
    RejectStanza-> "reject" ":" ident ([newline] [indent] ident])*
    StateStanze-> "state" ":" ident ([newline] [indent] StateTransition)+
    StateTransition-> (symbol|special) (symbol|special) action ident
    action-> l|r|n|L|R|N
    symbol-> \w[!$-/]     # note: a tape symbol
    special-> ":"
    ident-> \w+           # note: name of a state

    """

    def __init__(self):
        self.start_state = "" # is an state identifier
        self.accept_states = set() # is a set of state identifiers
        self.reject_states = set() # is a set of state identifiers
        self.transitions = {} # is a map of (state,symbol):(state,symbol,action)
        self.current_state = "" 
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.tape = ['_']  # is a list of symbols
        self.position = 0
        self.verbose = 0
        self.result = 0

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        self.tape =  ['_' if symbol==':' or symbol==' ' else 
                          symbol for symbol in tape_string]

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action."
        x = (state_from, read_symbol)
        if x in self.transitions:
            # return something instead, nobody likes a chatty program
            return "WARNING: multiple outgoing states not allowed for DFA's."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = '_'
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        elif (c_s,':') in self.transitions:
            # wildcard code
            (new_state, symbol, action ) = self.transitions[(c_s,':')]
        else:
            # here we implement a rejection of convenience, if there is
            # no transition, tansition target is (:, n, A_REJECT_STATE)
            self.reason = 1
            return False
        
        # wildcard code
        if symbol==':':
            symbol = self.tape[self.position]

        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = '_'
        if action == 'n':
            pass
   
        if shout or self.verbose == TuringMachine.verbose_levels['explain']:
            self.print_tape()
        if self.verbose == TuringMachine.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
            
        self.step_counter += 1
        return True

    def compute_tm(self,tape_string,step_limit=0,verbose='none'):
        self.verbose = TuringMachine.verbose_levels[verbose]
        self.result = 0
        self.restart(tape_string)
        #if self.verbose == TuringMachine.verbose_levels[verbose]:
            #self.print_tape()
        step = 0
            
        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if not res:
                # missing transition is considered a reject
                return False
            step += 1
            if step > step_limit:
                self.result = 2 
                return None
            
            if self.verbose == TuringMachine.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        if self.current_state in self.accept_states:
            return True
        return False

    def print_tape(self):
        t, p = self.tape, self.position
        s = ''.join(t[:p] + ['['] + [t[p]] + [']'] + t[p+1:])
        print(f'{self.current_state}:\t{s}')
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
    
    @classmethod
    def help(cls):
        print('The verbose levels are:')
        for level in cls.verbose_levels:
            print(f'\t{cls.verbose_levels[level]}: {level}')
        print()
        print('The grammar for the Turing Machine description is:')
        print(cls.grammar)
        
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|[!$-/:])\s+(\w|[!$-/:])\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res, f'line number {line_no}')
                                return False
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                return False
                break

            line_no += 1
        return True

### end class MachineParser



In [3]:

def create_and_test_turing_machine(tm_description, test_cases,verbose='none'):
    tm = TuringMachine()
    MachineParser.turing(tm,tm_description)
 
    print("\n*** TEST RUNS ***")

    for s in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(s,step_limit=10*(len(s)+5)**2,verbose=verbose)
        if res==True:
            print(f'ACCEPT input {s}\n')
        elif res==False:
            print(f'REJECT input {s}\n')
        else:
            print(f'ERROR on input {s}: {TuringMachine[tm.result]}')
            
    print("\n\n*** RUN COMPLETE ***\n\n")

# TuringMachine.help()

def create_and_test_turing_machine_flex(tm_description, test_cases,verbose='none'):
    tm = TuringMachine()
    MachineParser.turing(tm,tm_description)
 
    print("\n** TEST RUNS **")

    for s, resp in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(s,step_limit=10*(len(s)+5)**2,verbose=verbose)
        if res != resp:
            print(f'REJECT input \'{s}\' - expected {resp}\n')
            break
    print("\n\n** RUN COMPLETE ** successfully completed {0} cases\n\n".format(len(test_cases)))

## Exercise A

In [3]:

# Turing Machine M3, Sipser 3ird ed page 174, Sipser 2nd ed page 146

tm_M3 = """# The language of multiplication
# a^i b^j c^k, i,j,k >=1, and k = i*j

# a student assignment 

start: q0
accept: qA
reject: qR

state: q0
    a A r q15
    
state: q15
    a : r q15
    b : r q1
    
state: q1
    b : r q1
    c : r q2

state: q2
    c : r q2
    _ : l q3
    
state: q3
    a : l q3
    b : l q3
    c : l q3
    A a l q4

state: q4
    a x r q5
    b : r q4
    C : r q4
    _ : r qA

state: q5
    a : r q5
    C : l q8
    b B r q6

state: q6
    b : r q6
    C : r q6
    c C l q7
    _ : l qR

state: q7
    C : l q7
    b : l q7
    B : r q5
    
state: q8
    B b l q8
    a : l q8
    x : r q4

    
"""


tm_M3_test = [
    "",
    "cabc",
    "abc",
    "aaaaabccccc",
    "abbbbbccccc",
    "aabcc",
    "aabbbcccccc",
    "aabbcccc",
    "abbc",
    "aaabc",
    "aabbccccc",
    "ab",
    "bc",
    "ac",
    "abca",
    "abcb", 
    "abcc"
]

#create_and_test_turing_machine(tm_M3,tm_M3_test,verbose='none') 

from itertools import product

def tm_M3_generator (count: int) -> list:
    return [(''.join(['a'] * i + ['b'] * j + ['c'] * k), i * j == k) for i in range(1, count + 1) for j in range(1, (count + 1) - i) for k in range(1, (count + 1) - (i + j))]



exhaustive_tm_M3 = tm_M3_generator(100)
create_and_test_turing_machine_flex(tm_M3, exhaustive_tm_M3)


** TEST RUNS **


** RUN COMPLETE ** successfully completed 161700 cases




## Exercise B

In [6]:

# Turing Machine M4, Sipser 3ird ed page 175, Sipser 2nd ed page 147

tm_M4 = """# The language of distinct elements
# &x1&x2&...&xk where each xi in {0,1}*, and xi != xj for each i != j

# student assignment 
# note: the book says to place a mark on top of a &. let the tape symbol for a
# "marked" & be a %.

start: Start
accept: qA
reject: qR

state: Start               Mark & to S
    _ : r qA
    & S r readToA
    
state: readToA            Read to &, mark as %
    0 : r readToA
    1 : r readToA
    _ : r qA
    & % r readNextChar
    
state: readNextChar      Read right-string's 0, 1, or blank and mark it. Move to their machines.
    0 x l Z0             
    1 y l O0
    _ : l B0
    x : r readNextChar
    y : r readNextChar
    & S l B0
    S : l B0
    
state: readToP           Read through everything to get to %. You should never encounter x's or y's.
    0 : r readToP
    1 : r readToP
    & : r readToP
    % : r readNextChar
    
state: checkNextString   Check next right string. Change & to % and then clean current. If next & is %, change to S and reset all
    0 : r checkNextString   
    1 : r checkNextString
    x : r checkNextString
    y : r checkNextString
    & % r checkNextStringNoChange
    % S l undoMarks
    
state: checkNextStringNoChange   Run up to the rightmost %
    0 : r checkNextStringNoChange   
    1 : r checkNextStringNoChange
    & : r checkNextStringNoChange
    % : r cleanCurrent2
    
state: cleanCurrent      Read up to rightmost %
    0 : r cleanCurrent
    1 : r cleanCurrent
    & : r cleanCurrent
    % : r cleanCurrent2
    S : l readNextChar
    
state: cleanCurrent2     Turn any x -> 0 and y -> 1 until next &, then backTo% and read again
    x 0 r cleanCurrent2
    y 1 r cleanCurrent2
    0 : l backToP
    1 : l backToP
    & : l backToP
    _ : l backToP
    S : l backToP

state: backToP           After cleaning, run back to % to start new comparison
    0 : l backToP
    1 : l backToP
    % : r readNextChar
    
state: readToS           After undoing everything, run back to temp S somewhere in string
    0 : r readToS
    1 : r readToS
    S & r cleanCurrentAndMove
    & : r readToS
    
state: cleanCurrentAndMove      Read up to next rightmost &, signal it as next, then undo to the left
    x 0 r cleanCurrentAndMove
    y 1 r cleanCurrentAndMove
    0 : r cleanCurrentAndMove
    1 : r cleanCurrentAndMove
    & % r readNextChar
    S % r readNextChar
    _ : l qA

state: Z0                Left back through self to own %, then further back
    x : l Z0
    y : l Z0
    % : l Z1
    
state: Z1                Back through all & to leftmost S or %, remembering 0
    x : r Z2
    y : r Z2
    % : r Z2
    S : r Z2
    & : l Z1
    0 : l Z1
    1 : l Z1
    
state: Z2                If matches 0, run to rightmost %. If not: (1) check next right string, (%) checked all strings - change % to S and reset all, (&) clean rightmost % string
    0 x r readToP
    1 : r checkNextString
    % S l undoMarks
    & % r cleanCurrent

state: O0                Left back through self to own %, then further back
    x : l O0
    y : l O0
    % : l O1
    
state: O1                Back through all & to leftmost S or %, remembering 1
    x : r O2
    y : r O2
    % : r O2
    S : r O2
    & : l O1
    0 : l O1
    1 : l O1
    
state: O2                If matches 1, run to rightmost %. If not: (o) check next right string, (%) checked all strings - change % to S and reset all, (&) clean rightmost % string
    x : r O2
    y : r O2
    1 y r readToP
    0 : r checkNextString
    % S l undoMarks
    & % r cleanCurrent

state: B0                Left back through self to own %, then further back
    x : l B0
    y : l B0
    % : l B1
    
state: B1     Back through all & to leftmost S or %, remembering _
    x : l B1
    y : l B1
    % : r B2
    S : r B2
    & : l B1
    0 : l B1
    1 : l B1
    
state: B2      If there is no 0 or 1 between % and &, fail.
    % : r qR 
    x : r B2
    y : r B2
    0 : r checkNextString
    1 : r checkNextString
    & : r qR

state: undoMarks     Start by running back and undoing everything to S, then move to new machine that right to next S
    x 0 l undoMarks
    y 1 l undoMarks
    0 : l undoMarks
    1 : l undoMarks
    % & l undoMarks
    & : l undoMarks
    S : r readToS
    _ : r qA


"""

tm_M4_test = [
    "&01&01"
]

create_and_test_turing_machine(tm_M4,tm_M4_test,verbose='explain')

    


*** TEST RUNS ***
readToA:	S[0]1&01
readToA:	S0[1]&01
readToA:	S01[&]01
readNextChar:	S01%[0]1
Z0:	S01[%]x1
Z1:	S0[1]%x1
Z1:	S[0]1%x1
Z1:	[S]01%x1
Z2:	S[0]1%x1
readToP:	Sx[1]%x1
readToP:	Sx1[%]x1
readNextChar:	Sx1%[x]1
readNextChar:	Sx1%x[1]
O0:	Sx1%[x]y
O0:	Sx1[%]xy
O1:	Sx[1]%xy
O1:	S[x]1%xy
O2:	Sx[1]%xy
readToP:	Sxy[%]xy
readNextChar:	Sxy%[x]y
readNextChar:	Sxy%x[y]
readNextChar:	Sxy%xy[_]
B0:	Sxy%x[y]_
B0:	Sxy%[x]y_
B0:	Sxy[%]xy_
B1:	Sx[y]%xy_
B1:	S[x]y%xy_
B1:	[S]xy%xy_
B2:	S[x]y%xy_
B2:	Sx[y]%xy_
B2:	Sxy[%]xy_
qR:	Sxy%[x]y_
REJECT input &01&01



*** RUN COMPLETE ***




## Exercise C

Write and test TM's for the following languages over the alphabet { 0, 1 },

1. all strings w that contain equal numbers of 0s and 1s
1. all strings w that contain twice as many 0s as 1s
1. all strings w that do not contain twice as many 0s as 1s.


In [5]:
tm_3_8_a = """# exercise 3.8(a) in sipser

start: Start
accept: qA
reject: qR

state: Start               Mark & to S
    0 & r lfone
    1 & r lfzero
    _ : r qA
    
state: lfone
    0 : r lfone
    1 x l backToA
    x : r lfone

state: lfzero
    0 x l backToA
    1 : r lfzero
    x : r lfzero
    
state: backToA
    0 : l backToA
    1 : l backToA
    x : l backToA
    & : r lfnext

state: lfnext
    x & r lfnext
    0 x l bgone
    1 x l bgzero
    _ : l qA

state: bgone
    0 : l bgone
    1 x l backToA
    & : r lfone
    
state: bgzero
    0 x l backToA
    1 : l bgzero
    & : r lfzero
    
    
"""
tm_3_8_atest = [
    "00",
    "01",
    "10",
    "11",
    "0000",
    "0110",
    "1001",
    "0101",
    "1010",
    "0000",
    "1111",
    "1110",
    "0111",
    "1011",
    "1101"
    
]

#create_and_test_turing_machine(tm_3_8_a,tm_3_8_atest,verbose='none')

tm_3_8_b = """# exercise 3.8(b) in sipser

start: Start
accept: qA
reject: qR

state: Start               Mark & to S
    0 & r zs
    1 & r os
    _ : r qA

state: zs
    0 o r z2
    1 W r z3
    _ : l qR
    
state: z2
    0 : r z2
    1 W l toS
    _ : l qR
    
state: z3
    0 o l toS
    1 : r z3
    _ : r qR
    
state: os
    0 o r o2
    1 : r os
    _ : r qR
    
state: o2
    0 o l toS
    1 : r o2
    _ : r qR
    
state: toS
    0 : l toS
    1 : l toS
    o : l toS
    W : l toS
    & : r lfone

state: lfone
    0 : r lfone
    o : r lfone
    W : r lfone
    1 W l toStart0
    _ : l finalCheck
    
state: toStart0
    o : l toStart0
    W : l toStart0
    1 : l toStart0
    0 : l toStart0
    & : r lfzero
    
state: lfzero
    0 o r lfzero2
    1 : r lfzero
    o : r lfzero
    W : r lfzero
    
state: lfzero2
    0 o l toS
    1 : r lfzero2
    o : r lfzero2
    W : r lfzero2
    
state: finalCheck
    o : l finalCheck
    W : l finalCheck
    1 : l qR
    0 : l qR
    & : r finalCheck2
    
state: finalCheck2
    o : r finalCheck2
    W : r finalCheck2
    _ : r qA
"""

tm_3_8_c = """# exercise 3.8(c) in sipser

start: Start
accept: qA
reject: qR

state: Start               Mark & to S
    0 & r zs
    1 & r os
    _ : r qR

state: zs
    0 o r z2
    1 W r z3
    _ : l qA
    
state: z2
    0 : r z2
    1 W l toS
    _ : l qA
    
state: z3
    0 o l toS
    1 : r z3
    _ : r qA
    
state: os
    0 o r o2
    1 : r os
    _ : r qA
    
state: o2
    0 o l toS
    1 : r o2
    _ : r qA
    
state: toS
    0 : l toS
    1 : l toS
    o : l toS
    W : l toS
    & : r lfone

state: lfone
    0 : r lfone
    o : r lfone
    W : r lfone
    1 W l toStart0
    _ : l finalCheck
    
state: toStart0
    o : l toStart0
    W : l toStart0
    1 : l toStart0
    0 : l toStart0
    & : r lfzero
    
state: lfzero
    0 o r lfzero2
    1 : r lfzero
    o : r lfzero
    W : r lfzero
    _ : r qA
    
state: lfzero2
    0 o l toS
    1 : r lfzero2
    o : r lfzero2
    W : r lfzero2
    _ : r qA
    
state: finalCheck
    o : l finalCheck
    W : l finalCheck
    1 : l qA
    0 : l qA
    & : r finalCheck2
    
state: finalCheck2
    o : r finalCheck2
    W : r finalCheck2
    _ : r qR
"""

tm_3_8_ctest = [
    "101110010101101"
    
]

#create_and_test_turing_machine(tm_3_8_c,tm_3_8_ctest,verbose='explain')

from itertools import product
import types

def string_generation_count_1 (count: int) -> list:
    if (count <= 0):
        return [('', 0)]
    return list(set([(''.join([item[0][0], item[1]]), item[0][1] if item[1] != '1' else item[0][1] + 1) for item in product(string_generation_count_1(count - 1), ['', '0', '1'])]))
    
def string_generation_wrapper (predicate: types.LambdaType, count: int = 0, items: list = None) -> list:
    return [(item[0], predicate(item[0], item[1])) for item in (string_generation_count_1(count) if items is None else items)]

strings = string_generation_count_1(18)
    
exhaustive_a = string_generation_wrapper(lambda s, ones_count: len(s) == ones_count * 2, items=strings)
exhaustive_b = string_generation_wrapper(lambda s, ones_count: len(s) == ones_count * 3, items=strings)
exhaustive_c = string_generation_wrapper(lambda s, ones_count: len(s) != ones_count * 3, items=strings)

create_and_test_turing_machine_flex(tm_3_8_a, exhaustive_a)
create_and_test_turing_machine_flex(tm_3_8_b, exhaustive_b)
create_and_test_turing_machine_flex(tm_3_8_c, exhaustive_c)


** TEST RUNS **


** RUN COMPLETE ** successfully completed 524287 cases



** TEST RUNS **


** RUN COMPLETE ** successfully completed 524287 cases



** TEST RUNS **


** RUN COMPLETE ** successfully completed 524287 cases


